In [189]:
from binance.client import Client
import pandas as pd
import plotly.graph_objects as go
from pon import api,secret
import numpy as np
from datetime import timedelta
import plotly.io as pio
import schedule
import time
from plotly.subplots import make_subplots
import pandas_ta as ta
import matplotlib.pyplot as plt
from scipy.stats import linregress
import warnings
warnings.filterwarnings("ignore")


In [190]:
def get_price_data_binance(ticker:str,limit:int,start_str:str, end_str:str,interval = Client.KLINE_INTERVAL_1DAY)->pd.DataFrame:
    client = Client(api, secret)
    df = pd.DataFrame(client.get_historical_klines(symbol=ticker, interval=interval,start_str=start_str, end_str=end_str))
    df.columns=['date','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df = df.drop(['close_time','d1','d2','d3','d4','d5'],axis=1)
    df['date'] = pd.to_datetime(df['date']*1000000)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    return df
    
def get_list_of_imbalances(df:pd.DataFrame)->list:
    up_imbalances = []
    down_imbalances = []
    df['ATR'] = ta.atr(df['high'],df['low'],df['close'],14)
    for i in range(5, len(df)-2):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        #if (df.iloc[i-1]['close'] - df.iloc[i-1]['open'])/3 < (next_candle['low'] - prev_candle['high']):
        if (prev_candle['high'] < next_candle['low']) and df.iloc[i]['ATR']/(next_candle['low'] - prev_candle['high'])<20:
            middle = round((prev_candle['high']+next_candle['low'])/2,2)
            if df.iloc[i+1]['low'] > middle:
                if df.iloc[i+2]['low'] > middle:
                    if (df.iloc[i-1]['close']-next_candle['low']) > (next_candle['low']-middle) or (next_candle['close']-next_candle['low']) > (next_candle['low']-middle):
                        up_imbalances.append([middle, 
                        prev_candle['high'],
                        df.iloc[i-1]['date'],
                        prev_candle['low']])

        #if (df.iloc[i-1]['open'] - df.iloc[i]['open'])/3 < (prev_candle['low'] - next_candle['high']):
        if (prev_candle['low'] > next_candle['high']) and df.iloc[i]['ATR']/(prev_candle['low'] - next_candle['high'])<20:
            middle = round((prev_candle['low']+next_candle['high'])/2,2)
            if df.iloc[i+1]['high'] < middle:
                if df.iloc[i+2]['high'] < middle:
                    if (next_candle['high']-df.iloc[i-1]['close']) > (middle-next_candle['high']) or (next_candle['high']-next_candle['close']) > (middle-next_candle['high']):
                        down_imbalances.append([middle,
                        prev_candle['low'],
                        df.iloc[i-1]['date'],
                        prev_candle['high']])

    return up_imbalances, down_imbalances

def check_within_imbalances(df:pd.DataFrame, up_imbalances:list, down_imbalances:list)->pd.DataFrame:
    df['within_up_imbalance'] = False
    df['within_down_imbalance'] = False
    df['stop'] = 0
    up_imbalancesCOPY = up_imbalances.copy()
    down_imbalancesCOPY = down_imbalances.copy()
    for i in range(2, len(df)):
        for imbalance in up_imbalancesCOPY:
            # if df.iloc[i]['low'] < imbalance[0] and df.iloc[i]['close'] > imbalance[1] and df.iloc[i]['date'] > imbalance[2]:
            if hammer_up(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date'] > imbalance[2]:
                if (df.iloc[i]['low']<imbalance[0] and df.iloc[i]['close']>imbalance[0] and df.iloc[i]['open']>imbalance[0]):
                    df.loc[i, 'within_up_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['low']
                    
            if hammer_up(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date'] > imbalance[2]:
                if (df.iloc[i]['low']<imbalance[1] and df.iloc[i]['close']>imbalance[1] and df.iloc[i]['open']>imbalance[1]):
                    df.loc[i, 'within_up_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['low']

        for imbalance in down_imbalancesCOPY:
            # if df.iloc[i]['high'] > imbalance[0] and df.iloc[i]['close'] < imbalance[1] and df.iloc[i]['date'] > imbalance[2]:
            if falling_star_down(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date']>imbalance[2]:
                if df.iloc[i]['high']>imbalance[0] and df.iloc[i]['close']<imbalance[0] and df.iloc[i]['open']<imbalance[0]:
                    df.loc[i, 'within_down_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['high']
                    
            if falling_star_down(df.loc[i, ['open', 'high', 'low', 'close']]) and df.iloc[i]['date']>imbalance[2]:
                if df.iloc[i]['high']>imbalance[1] and df.iloc[i]['close']<imbalance[1] and df.iloc[i]['open']<imbalance[1]:
                    df.loc[i, 'within_down_imbalance'] = True
                    df.loc[i, 'stop'] = df.iloc[i]['high']

        for imbalance in up_imbalancesCOPY:
            if df.iloc[i-2]['close'] < imbalance[1] and df.iloc[i-2]['date'] > imbalance[2]:
                up_imbalancesCOPY.remove(imbalance)
        
        for imbalance in down_imbalancesCOPY:
            if df.iloc[i-2]['close'] > imbalance[1] and df.iloc[i-2]['date'] > imbalance[2]:
                down_imbalancesCOPY.remove(imbalance)

    return df

def delete_up_imbalance(up_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)-10):
        for imbalance in up_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2]  and (candle['close'] < imbalance[0] or candle['low'] < imbalance[0]):
                up_imbalances.remove(imbalance)
    return up_imbalances[-3:]

def delete_down_imbalance(down_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)-10):
        for imbalance in down_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2] and (candle['close'] > imbalance[0] or candle['high'] > imbalance[0]):
                down_imbalances.remove(imbalance)
    
    return down_imbalances[-3:]

def get_list_of_levels(df:pd.DataFrame)->list:
    levels = []
    for i in range(len(df)):
        levels.append([df.iloc[i]['high'],df.iloc[i]['date']])
        levels.append([df.iloc[i]['low'],df.iloc[i]['date']])
        
    return levels

def add_moving_average(df:pd.DataFrame)->pd.DataFrame:
    
    df['SMA9'] = df['Close'].rolling(9).mean()
    df['SMA9slope'] = df['SMA9'].pct_change()*1000
    
    df['SMA12'] = df['Close'].rolling(12).mean()
    df['SMA12slope'] = df['SMA12'].pct_change()*1000
    
    df['SMA25'] = df['Close'].rolling(25).mean()
    df['SMA25slope'] = df['SMA25'].pct_change()*1000
    
    df['SMA200'] = df['Close'].rolling(200).mean()
    df['SMA200slope'] = df['SMA200'].pct_change()*1000
    df['SMA200sum'] = df['SMA200slope'].rolling(50).sum()
    
    df['crossunder'] = ((df['SMA12'].shift(1) < df['SMA25'].shift(1)) & (df['SMA12'] > df['SMA25'])).astype(int)
    df['crossunder'] = df['crossunder'].apply(lambda x: 1 if x else 0)



    

    # df['up_reversal'] = ((df['SMAslope'].shift(1) < 0) & (df['SMAslope'] > 0))
    # df['down_reversal'] = ((df['SMAslope'].shift(1) > 0) & (df['SMAslope'] < 0)) 

    return df

def add_rsx_HF(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    df['RSXslope'] = df['RSX'].pct_change()*100
    df['RSX_up_reversal'] = ((df['RSXslope'].shift(1) < 0) & (df['RSXslope'] > 0)) & (df['RSX'] < 30)
    df['RSX_down_reversal'] = ((df['RSXslope'].shift(1) > 0) & (df['RSXslope'] < 0)) & (df['RSX'] > 70)

    up_reversals = []
    down_reversals = []
    # df['hidden_up_divergence'] = False
    # df['hidden_down_divergence'] = False
    df['classic_up_divergence'] = False
    df['classic_down_divergence'] = False

    for i in range(0, len(df)):
        if df.iloc[i]['RSX_up_reversal']:
            up_reversals.append(min((df.iloc[i]['low'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['low'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['low'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['low'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['low'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['low'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if df.iloc[i]['RSX_down_reversal']:
            down_reversals.append(max((df.iloc[i]['high'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['high'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['high'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['high'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['high'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['high'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if up_reversals:
            if df.iloc[i]['date']>up_reversals[-1][2]:
                if df.iloc[i]['low'] < up_reversals[-1][0]:
                    if df.iloc[i]['RSX'] > up_reversals[-1][1]:
                        df.loc[i, 'classic_up_divergence'] = True

        if down_reversals:
            if df.iloc[i]['date']>down_reversals[-1][2]:
                if df.iloc[i]['high'] > down_reversals[-1][0]:
                    if df.iloc[i]['RSX'] < down_reversals[-1][1]:
                        df.loc[i, 'classic_down_divergence'] = True

        # if (up_reversals and down_reversals):
        #     for up_reversal in up_reversals[-5:]:
        #         if (up_reversal[0] < df.iloc[i]['low']) and (up_reversal[1] > df.iloc[i]['RSX']) and up_reversal[2] < df.iloc[i]['date']:
        #             df.loc[i, 'hidden_up_divergence'] = True
                
        #     for down_reversal in down_reversals[-5:]:
        #         if (down_reversal[0] > df.iloc[i]['high']) and (down_reversal[1] < df.iloc[i]['RSX']) and down_reversal[2] < df.iloc[i]['date']:
        #             df.loc[i, 'hidden_down_divergence'] = True
    return df

def add_rsx_LF(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    df['RSXslope'] = df['RSX'].pct_change()*100
    df['RSX_up_reversal'] = ((df['RSXslope'].shift(1) < 0) & (df['RSXslope'] > 0)) & (df['RSXslope'].shift(1).rolling(5).sum()<-6)
    df['RSX_down_reversal'] = ((df['RSXslope'].shift(1) > 0) & (df['RSXslope'] < 0)) & (df['RSXslope'].shift(1).rolling(5).sum()>6)

    up_reversals = []
    down_reversals = []
    df['classic_up_divergence'] = False
    df['classic_down_divergence'] = False

    for i in range(len(df) - 200, len(df)):
        if df.iloc[i]['RSX_up_reversal']:
            up_reversals.append(min((df.iloc[i]['low'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['low'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['low'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['low'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['low'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['low'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if df.iloc[i]['RSX_down_reversal']:
            down_reversals.append(max((df.iloc[i]['high'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['high'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['high'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['high'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['high'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['high'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))
                
        if (up_reversals and down_reversals):
            for up_reversal in up_reversals[-1:]:    
                if (up_reversal[0] > df.iloc[i]['low']) and (up_reversal[1] < df.iloc[i]['RSX']) and up_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'classic_up_divergence'] = True

            for down_reversal in down_reversals[-1:]:       
                if (down_reversal[0] < df.iloc[i]['high']) and (down_reversal[1] > df.iloc[i]['RSX']) and down_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'classic_down_divergence'] = True
                
    return df

def add_rsx(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    leftcounter = 5
    rightcounter = 5
    df['RSX_up_reversal'] = 110
    df['RSX_down_reversal'] = -10
    df['local_low'] = 100_000
    df['local_high'] = -10
    df['down_classic_divergence'] = False
    df['up_classic_divergence'] = False
    for i in range(leftcounter,len(df)-rightcounter):
        if min(df.loc[i-leftcounter:i-1,'RSX'])>=df.iloc[i]['RSX'] and min(df.loc[i+1:i+rightcounter,'RSX'])>=df.iloc[i]['RSX']:
            # if df.iloc[i+1]['close']>df.iloc[i+1]['open'] and df.iloc[i+2]['close']>df.iloc[i+2]['open']:
                df.loc[df.index[i],'RSX_up_reversal'] = df.loc[df.index[i],'RSX']
                df.loc[df.index[i],'local_low'] = min(df.loc[i-10:i,'low'])
            
        if max(df.loc[i-leftcounter:i-1,'RSX'])<=df.iloc[i]['RSX'] and max(df.loc[i+1:i+rightcounter,'RSX'])<=df.iloc[i]['RSX']:
            # if df.iloc[i+1]['close']<df.iloc[i+1]['open'] and df.iloc[i+2]['close']<df.iloc[i+2]['open']:
                df.loc[df.index[i],'RSX_down_reversal'] = df.loc[df.index[i],'RSX']
                df.loc[df.index[i],'local_high'] = max(df.loc[i-10:i,'high'])
    leftcounter = 15
    for i in range(leftcounter,len(df)):
        if max(df.loc[i-leftcounter:i-5,'RSX_down_reversal'])>df.loc[df.index[i],'RSX']:
            if max(df.loc[i-leftcounter:i-5,'local_high'])<df.loc[df.index[i],'high']:
                df.loc[df.index[i],'down_classic_divergence'] = True
        if min(df.loc[i-leftcounter:i-5,'RSX_up_reversal'])<df.loc[df.index[i],'RSX']:
            if min(df.loc[i-leftcounter:i-5,'local_low'])>df.loc[df.index[i],'low']:
                df.loc[df.index[i],'up_classic_divergence'] = True
    return df

def three_candle_up_fractal(left:list,medium:list,right:list):
    #ohlc - 0123
    if left[0] > left[3] and right[0] < right[3] and left[0] > medium[3] and left[0] < right[3] and (left[0]-left[3])*2 > (right[3]-right[0]):
        return True
        
def three_candle_down_fractal(left:list,medium:list,right:list):
    #ohlc - 0123
    if left[0] < left[3] and right[0] > right[3] and left[0] < medium[3] and left[0] > right[3] and (left[3]-left[0])*2 > (right[0]-right[3]):
        return True
 
def add_MACD(df:pd.DataFrame)->pd.DataFrame:
    macd = pd.DataFrame(ta.macd(df['close']))
    macd = macd.drop('MACDh_12_26_9',axis=1)
    macd = macd.rename(columns={'MACD_12_26_9':'MACD','MACDs_12_26_9':'MACDs'})
    df['MACDs'] = macd['MACDs']
    df['MACD'] = macd['MACD']
    # df = pd.concat([df, macd])
    return df
  
def add_bb(df:pd.DataFrame)->pd.DataFrame:
    bb = ta.bbands(df['close'])
    bb = bb.drop(['BBB_5_2.0','BBP_5_2.0'],axis=1)
    bb = bb.rename(columns={'BBL_5_2.0':'BBL','BBM_5_2.0':'BBM','BBU_5_2.0':'BBU'})
    df['BBL'] = bb['BBL'].round(2)
    df['BBM'] = bb['BBM'].round(2)
    df['BBU'] = bb['BBU'].round(2)
    return df
    
def channels(df:pd.DataFrame):
    watchleft = 60
    watchleftline = 30
    watchright = 30
    watchrightline = 5

    counter = 0
    for i in range(watchleft,len(df)-watchright):
        down_values = df['down_reversal'].iloc[i-watchleftline:i-1]  # get the last 60 values in the column
        down_values = down_values[down_values != 0]  # remove any values equal to 100_000
        down_indexes = down_values.index.to_list()
        down_values = down_values.to_list()
        
        up_values = df['up_reversal'].iloc[i-watchleftline:i-1]  # get the last 60 values in the column
        up_values = up_values[up_values != 100_000]  # remove any values equal to 100_000
        up_indexes = up_values.index.to_list()
        up_values = up_values.to_list()

        if len(up_indexes)>2 and len(down_indexes)>2:
            # Вычисляем коэффициенты k и b
            kdown, bdown = np.polyfit(down_indexes, down_values, deg=1)
            kdown, bdown = round(kdown,2),round(bdown,2)
            
            kup, bup = np.polyfit(up_indexes, up_values, deg=1)
            kup, bup = round(kup,2),round(bup,2)
            
            if df.iloc[i]['open']<df.iloc[i]['close']:
                if (df.iloc[i]['close']>kdown*i+bdown and df.iloc[i]['low']<kdown*i+bdown) and abs(kdown)<30 and abs(kup)<30:
                    if (kdown<-1 and kup>1) or (kdown<0 and kup<0) or (kdown>0 and kup>0):
                        fig = go.Figure(data=[go.Candlestick(x=df['date'][i-watchleft:i+watchright],
                                            open=df['open'][i-watchleft:i+watchright],
                                            high=df['high'][i-watchleft:i+watchright],
                                            low=df['low'][i-watchleft:i+watchright],
                                            close=df['close'][i-watchleft:i+watchright])])
                        y = [i*kdown+bdown for i in range(i-watchleftline,i+watchrightline)]
                        fig.add_trace(go.Scatter(x=df.loc[i-watchleftline:i+watchrightline,'date'], y=y, mode='lines',line=dict(color='green')))
                        y = [i*kup+bup for i in range(i-watchleftline,i+watchrightline)]
                        fig.add_trace(go.Scatter(x=df.loc[i-watchleftline:i+watchrightline,'date'], y=y, mode='lines',line=dict(color='red')))     
                        fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['close']],mode='markers', marker=dict(size=4, color='white')))
                        fig.update_layout(title=f'kup-{kup}-kdown{kdown}')
                        counter+=1
                        pio.write_image(fig, f'screens\\screen№{counter}.jpg')
                        
            if df.iloc[i]['open']>df.iloc[i]['close']:
                if df.iloc[i]['close']<kup*i+bup and df.iloc[i]['high']>kup*i+bup and abs(kdown)<30 and abs(kup)<30:
                    if (kdown<-1 and kup>1) or (kdown<0 and kup<0) or (kdown>0 and kup>0):
                        fig = go.Figure(data=[go.Candlestick(x=df['date'][i-watchleft:i+watchright],
                                            open=df['open'][i-watchleft:i+watchright],
                                            high=df['high'][i-watchleft:i+watchright],
                                            low=df['low'][i-watchleft:i+watchright],
                                            close=df['close'][i-watchleft:i+watchright])])
                        y = [i*kdown+bdown for i in range(i-watchleftline,i+watchrightline)]
                        fig.add_trace(go.Scatter(x=df.loc[i-watchleftline:i+watchrightline,'date'], y=y, mode='lines',line=dict(color='green')))
                        y = [i*kup+bup for i in range(i-watchleftline,i+watchrightline)]
                        fig.add_trace(go.Scatter(x=df.loc[i-watchleftline:i+watchrightline,'date'], y=y, mode='lines',line=dict(color='red')))     
                        fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['close']],mode='markers', marker=dict(size=4, color='white')))
                        fig.update_layout(title=f'kup-{kup}-kdown{kdown}')
                        counter+=1
                        pio.write_image(fig, f'screens\\screen№{counter}.jpg')
    
def add_EMA(period:int,df:pd.DataFrame)->pd.DataFrame:
    df['EMA'] = ta.ema(df['close'],period)
    return df
    
def add_atr(period:int,df:pd.DataFrame)->pd.DataFrame:
    df['ATR'] = ta.atr(df['high'],df['low'],df['close'],period)   
    return df 
    
def hammer_up(current:list):
    #ohlc - 0123
    #При первом множителе 2 винрейт почти такой же, но сделок больше, что увеличивает прибыль
    if current[0]>current[3]:
        if (current[0]-current[3])*3 < (current[1]-current[2]) and (current[1]-current[0])*4 < (current[0]-current[2]):
            return True
    if current[0]<current[3]:
        if (current[3]-current[0])*3 < (current[1]-current[2]) and (current[1]-current[3])*4 < (current[3]-current[2]):
            return True

def falling_star_down(current:list):
    if current[0]>current[3]:
        if (current[0]-current[3])*3 < (current[1]-current[2]) and (current[3]-current[2])*2 < (current[1]-current[3]):
            return True
    
    if current[0]<current[3]:
        if (current[3]-current[0])*3 < (current[1]-current[2]) and (current[0]-current[2])*2 < (current[1]-current[0]):
            return True
    
def down_engulfing(left:list,right:list):
    #ohlc - 0123
    if left[0]<left[3] and right[3]<left[0] and (left[3]-left[0])*2 > (right[0]-right[3]):
        if (right[1]-right[2])<(right[0]-right[3])*2:
            if (left[1]-left[2])<(left[3]-left[0])*2:
                return True
    
def up_engulfing(left:list,right:list):
    #ohlc - 0123
    if left[0]>left[3] and right[3]>left[0] and (left[0]-left[3])*2 > (right[3]-right[0]):
        if (right[1]-right[2])<(right[3]-right[0])*2:
            if (left[1]-left[2])<(left[0]-left[3])*2:
                return True

def local_reversals(df:pd.DataFrame)->pd.DataFrame:
    leftcounter = 2
    rightcounter = 2
    df['up_reversal'] = 100_000
    df['down_reversal'] = 0
    for i in range(leftcounter,len(df)-rightcounter):
        if min(df.loc[i-leftcounter:i-1,'low'])>=df.iloc[i]['low'] and min(df.loc[i+1:i+rightcounter,'low'])>=df.iloc[i]['low']:
            # if df.iloc[i+1]['close']>df.iloc[i+1]['open'] and df.iloc[i+2]['close']>df.iloc[i+2]['open']:
                df.loc[df.index[i],'up_reversal'] = df.loc[df.index[i],'low']
            
        if max(df.loc[i-leftcounter:i-1,'high'])<=df.iloc[i]['high'] and max(df.loc[i+1:i+rightcounter,'high'])<=df.iloc[i]['high']:
            # if df.iloc[i+1]['close']<df.iloc[i+1]['open'] and df.iloc[i+2]['close']<df.iloc[i+2]['open']:
                df.loc[df.index[i],'down_reversal'] = df.loc[df.index[i],'high']
    return df

def liquidity_reversals(df:pd.DataFrame):
    leftcounter = 2
    rightcounter = 2
    df['up_reversal'] = 100_000
    df['down_reversal'] = 0
    for i in range(leftcounter,len(df)-rightcounter):
        if min(df.loc[i-leftcounter:i-1,'low'])>=df.iloc[i]['low'] and min(df.loc[i+1:i+rightcounter,'low'])>=df.iloc[i]['low']:
            if bigwick(df.loc[i, ['open', 'high', 'low', 'close']]): # if df.iloc[i+1]['close']>df.iloc[i+1]['open'] and df.iloc[i+2]['close']>df.iloc[i+2]['open']:
                df.loc[df.index[i],'up_reversal'] = df.loc[df.index[i],'low']
            
        if max(df.loc[i-leftcounter:i-1,'high'])<=df.iloc[i]['high'] and max(df.loc[i+1:i+rightcounter,'high'])<=df.iloc[i]['high']:
            if bigwick(df.loc[i, ['open', 'high', 'low', 'close']]): # if df.iloc[i+1]['close']<df.iloc[i+1]['open'] and df.iloc[i+2]['close']<df.iloc[i+2]['open']:
                df.loc[df.index[i],'down_reversal'] = df.loc[df.index[i],'high']
    return df

def backtest(df: pd.DataFrame, buy_orders: list, sell_orders: list,depo:int,risk:int,depositchart=False,savecharts=True):
    # Initialize variables to store statistics
    candlesleft = 25
    candlesright = 14
    startdepo = depo
    counter = 0 
    win_buy_trades = 0
    lose_buy_trades = 0
    win_sell_trades = 0
    lose_sell_trades = 0
    depolist = [depo]
    RRlist = []
    # Iterate through buy orders
    for order in buy_orders:
        lose = depo*0.01*risk
        open_price = order[0]
        stop_loss = order[1]
        take_profit = order[2]
        open_index = order[3]
        for i in range(open_index, len(df)):
            if df.iloc[i]['low'] <= stop_loss:
                depo -= (1*lose+(lose*0.002))
                lose_buy_trades += 1
                depolist.append(depo)
                if savecharts:
                    fig = go.Figure(data=[go.Candlestick(x=df['date'][open_index-candlesleft:open_index+candlesright],
                            open=df['open'][open_index-candlesleft:open_index+candlesright],
                            high=df['high'][open_index-candlesleft:open_index+candlesright],
                            low=df['low'][open_index-candlesleft:open_index+candlesright],
                            close=df['close'][open_index-candlesleft:open_index+candlesright])])
                    fig.update_layout(title='Убыток')        
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[open_price],mode='markers', marker=dict(size=4, color='white')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[stop_loss],mode='markers', marker=dict(size=4, color='red')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[take_profit],mode='markers', marker=dict(size=4, color='green')))
                    counter+=1
                    pio.write_image(fig, f'screens\\screen№{counter}-Убыток.jpg')
                break
            elif df.iloc[i]['high'] >= take_profit:
                depo += (round((open_price - take_profit)/(stop_loss - open_price),2)*lose-(lose*0.002))
                win_buy_trades += 1
                depolist.append(depo)
                RRlist.append(round((open_price - take_profit)/(stop_loss - open_price),2))
                if savecharts:
                    fig = go.Figure(data=[go.Candlestick(x=df['date'][open_index-candlesleft:open_index+candlesright],
                            open=df['open'][open_index-candlesleft:open_index+candlesright],
                            high=df['high'][open_index-candlesleft:open_index+candlesright],
                            low=df['low'][open_index-candlesleft:open_index+candlesright],
                            close=df['close'][open_index-candlesleft:open_index+candlesright])])
                    fig.update_layout(title='Прибыль')        
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[open_price],mode='markers', marker=dict(size=4, color='white')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[stop_loss],mode='markers', marker=dict(size=4, color='red')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[take_profit],mode='markers', marker=dict(size=4, color='green')))
                    counter+=1
                    pio.write_image(fig, f'screens\\screen№{counter}-Прибыль.jpg')
                break
    
    # Iterate through sell orders
    for order in sell_orders:
        lose = depo*0.01*risk
        open_price = order[0]
        stop_loss = order[1]
        take_profit = order[2]
        open_index = order[3]
        
        # round((open_price - take_profit)/(stop_loss - open_price),2)
        
        for i in range(open_index, len(df)):
            if df.iloc[i]['high'] >= stop_loss:
                depo -= (1*lose+(lose*0.002))
                lose_sell_trades += 1
                depolist.append(depo)
                if savecharts:
                    fig = go.Figure(data=[go.Candlestick(x=df['date'][open_index-candlesleft:open_index+candlesright],
                            open=df['open'][open_index-candlesleft:open_index+candlesright],
                            high=df['high'][open_index-candlesleft:open_index+candlesright],
                            low=df['low'][open_index-candlesleft:open_index+candlesright],
                            close=df['close'][open_index-candlesleft:open_index+candlesright])])
                    fig.update_layout(title='Убыток')        
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[open_price],mode='markers', marker=dict(size=4, color='white')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[stop_loss],mode='markers', marker=dict(size=4, color='red')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[take_profit],mode='markers', marker=dict(size=4, color='green')))
                    counter+=1
                    pio.write_image(fig, f'screens\\screen№{counter}-Убыток.jpg')
                break
            elif df.iloc[i]['low'] <= take_profit:
                depo += (round((open_price - take_profit)/(stop_loss - open_price),2)*lose-(lose*0.002))
                win_sell_trades += 1
                depolist.append(depo)
                RRlist.append(round((open_price - take_profit)/(stop_loss - open_price),2))
                if savecharts:
                    fig = go.Figure(data=[go.Candlestick(x=df['date'][open_index-candlesleft:open_index+candlesright],
                            open=df['open'][open_index-candlesleft:open_index+candlesright],
                            high=df['high'][open_index-candlesleft:open_index+candlesright],
                            low=df['low'][open_index-candlesleft:open_index+candlesright],
                            close=df['close'][open_index-candlesleft:open_index+candlesright])])
                    fig.update_layout(title='Прибыль')        
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[open_price],mode='markers', marker=dict(size=4, color='white')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[stop_loss],mode='markers', marker=dict(size=4, color='red')))
                    fig.add_trace(go.Scatter(x=[df.iloc[open_index]['date']],y=[take_profit],mode='markers', marker=dict(size=4, color='green')))
                    counter+=1
                    pio.write_image(fig, f'screens\\screen№{counter}-Прибыль.jpg')
                break
    buy_winrate = 0
    sell_winrate = 0
    if len(buy_orders)>0:
        buy_winrate = round((win_buy_trades)/len(buy_orders)*100,2)
    if len(sell_orders)>0:
        sell_winrate = round((win_sell_trades)/len(sell_orders)*100,2)
    total_trades = win_buy_trades + win_sell_trades + lose_buy_trades + lose_sell_trades
    
    peak = depolist[0]
    max_drawdown = 0
    maxdrawdownpeak = 0
    if total_trades:
        winrate = round((win_buy_trades+win_sell_trades)/total_trades*100,2)
    else:
        winrate = 0
    for deposit in depolist:
        peak = max(peak, deposit)
        drawdown = round((peak - deposit)/peak*100,2)
        if max_drawdown < drawdown:
            max_drawdown = max(max_drawdown, drawdown)
            maxdrawdownpeak = peak
    RRfinale = 0
    if RRlist:
        RRfinale = round(sum(RRlist)/len(RRlist),2)
        
        
    print(f"""
        Total trades: {total_trades},
        Total win trades: {win_buy_trades + win_sell_trades}
        Total lose trades: {lose_buy_trades + lose_sell_trades}
        Total buy trades: {win_buy_trades + lose_buy_trades}
        Total sell trades: {win_sell_trades + lose_sell_trades}
        Winrate: {winrate}%
        Winrate buy trades: {buy_winrate}%
        Winrate sell trades: {sell_winrate}%
        Risk: {risk}%
        Avg RR: {RRfinale}
        Start deposit: {startdepo}$
        Deposit: {round(depo,3)}$
        Max Drawdown(%): {max_drawdown}%
        Max Drawdown($): {round(maxdrawdownpeak*0.01*max_drawdown,2)}$"""
    )
    if depositchart:
        fig = go.Figure(data=[go.Scatter(x=list(range(len(depolist))), y=depolist,mode='lines')])
        fig.show()

def bigwick(current:list):
    # ohlc
    if (current[1]-current[2])/2 > abs(current[3]-current[0]):
        return True

def saveworkingfuncs(df):
    # buy_orders = []
    # sell_orders = []

    # multiplier = 1.5
    # counter = 0
    # for i in range(35, len(df)-15):
    #     if min(df.loc[i-candlesleft:i-6,'up_reversal'])>=df.iloc[i-1]['low']:
    #         if min(df.loc[i-candlesleft:i-6,'up_reversal'])<df.iloc[i-1]['close'] and min(df.loc[i-candlesleft:i-6,'up_reversal'])<df.iloc[i-1]['open']:
    #             if min(df.loc[i-8:i-2,'low'])<df.iloc[i-1]['low']:
    #                 if bigwick(df.loc[i-1, ['open', 'high', 'low', 'close']]):
    #                     buy_orders.append((df.iloc[i]['open'], df.iloc[i-1]['low'],
    #                     round(df.iloc[i]['open'] + (df.iloc[i]['open']-df.iloc[i-1]['low'])*multiplier,2),
    #                     df.iloc[i]['date']))
        
    #     if max(df.loc[i-candlesleft:i-8,'down_reversal'])<=df.iloc[i-1]['high']:
    #         if max(df.loc[i-candlesleft:i-6,'down_reversal'])<=df.iloc[i-1]['high']:
    #             if max(df.loc[i-candlesleft:i-6,'down_reversal'])>df.iloc[i-1]['close'] and max(df.loc[i-candlesleft:i-6,'down_reversal'])>df.iloc[i-1]['open']:
    #                 if max(df.loc[i-8:i-2,'high'])<=df.iloc[i-1]['high']:
    #                     if bigwick(df.loc[i-1, ['open', 'high', 'low', 'close']]):
    #                         sell_orders.append((df.iloc[i]['open'], df.iloc[i-1]['high'],
    #                         round(df.iloc[i]['open'] - (df.iloc[i-1]['high']-df.iloc[i]['open'])*multiplier,2),
    #                         df.iloc[i]['date'])) 
        
    #     for i in range(watchleft, len(df)-1):
    # if (min(df.loc[i-watchleft:i-1,'up_reversal']) - df.iloc[i]['low'])>(df.iloc[i]['close'] - min(df.loc[i-watchleft:i-1,'up_reversal'])):
    #     if min(df.loc[i-watchleft:i-1,'up_reversal'])>df.iloc[i]['low'] and min(df.loc[i-5:i-1,'low'])>df.iloc[i]['low']:
    #         if min(df.loc[i-watchleft:i-1,'up_reversal'])<df.iloc[i]['close'] and min(df.loc[i-watchleft:i-1,'up_reversal'])<df.iloc[i]['open']:
    #             if (max(df.loc[i-5:i-1,'high'])-min(df.loc[i-watchleft:i-1,'up_reversal']))>(df.iloc[i]['high']-df.iloc[i]['low']):
    #                 buy_orders.append((df.iloc[i+1]['open'], df.iloc[i]['low'],
    #                 round(df.iloc[i+1]['open'] + (df.iloc[i+1]['open']-df.iloc[i]['low'])*multiplier,2),
    #                 i+1))
            
    # if (max(df.loc[i-watchleft:i-1,'down_reversal']) - df.iloc[i]['close'])<(df.iloc[i]['high'] - max(df.loc[i-watchleft:i-1,'down_reversal'])):
    #     if max(df.loc[i-watchleft:i-1,'down_reversal'])<df.iloc[i]['high'] and max(df.loc[i-5:i-1,'high'])<df.iloc[i]['high']:
    #         if max(df.loc[i-watchleft:i-1,'down_reversal'])>df.iloc[i]['close'] and max(df.loc[i-watchleft:i-1,'down_reversal'])>df.iloc[i]['open']:
    #             if (max(df.loc[i-watchleft:i-1,'down_reversal'])-min(df.loc[i-watchleft:i-1,'low']))>(df.iloc[i]['high']-df.iloc[i]['low']):
    #                 sell_orders.append((df.iloc[i+1]['open'], df.iloc[i]['high'],
    #                 round(df.iloc[i+1]['open'] - (df.iloc[i]['high']-df.iloc[i+1]['open'])*multiplier,2),
    #                 i+1)) 
    pass

In [191]:
# df_higher_tf = get_price_data_binance('BTCUSDT',1000,start_str='1 Jan, 2023', end_str='25 Jan, 2023',interval=Client.KLINE_INTERVAL_30MINUTE)
# df_lower_tf = get_price_data_binance('BTCUSDT',1000,start_str='1 Jan, 2023', end_str='25 Jan, 2023',interval=Client.KLINE_INTERVAL_15MINUTE)
# df_higher_tf2 = get_price_data_binance('BTCUSDT',1000,start_str='18 Dec, 2022', end_str='7 Jan, 2023',interval=Client.KLINE_INTERVAL_30MINUTE)
# df_higher_tf3 = get_price_data_binance('BTCUSDT',1000,start_str='28 Nov, 2022', end_str='18 Dec, 2022',interval=Client.KLINE_INTERVAL_30MINUTE)
# df_higher_tf4 = get_price_data_binance('BTCUSDT',1000,start_str='7 Nov, 2022', end_str='28 Nov, 2022',interval=Client.KLINE_INTERVAL_30MINUTE)
# enddf = pd.read_csv('enddf.csv')

# stoch = ta.stoch(high=df['High'], low=df['Low'], close=df['Close'])
#df_lower_tf = get_price_data_binance('BTCUSDT',1000,interval=Client.KLINE_INTERVAL_5MINUTE)

# df = get_price_data_binance('BTCUSDT',1000,start_str='06:00 27 Mar, 2023 ', end_str='14:00 27 Mar, 2023',interval=Client.KLINE_INTERVAL_5MINUTE)
df = get_price_data_binance('BTCUSDT',1000,start_str='1 Dec, 2022 ', end_str='28 Dec, 2022',interval=Client.KLINE_INTERVAL_15MINUTE)
# dflowertf = get_price_data_binance('BTCUSDT',1000,start_str='1 Apr, 2023 ', end_str='28 Apr, 2023',interval=Client.KLINE_INTERVAL_5MINUTE)

In [192]:
df = liquidity_reversals(df)
# df = add_rsx(14,df)

watchleftchart = 30
watchleft = 15
watchrightchart = 15
counter = 0
buy_orders = []
sell_orders = []
multiplier = 3
for i in range(watchleftchart, len(df)-watchrightchart):
    TFchange = 12
    if min(df.loc[i-watchleft:i-1,'up_reversal'])>=df.iloc[i]['low'] and min(df.loc[i-5:i-1,'low'])>df.iloc[i]['low']:
        if min(df.loc[i-watchleft:i-1,'up_reversal'])<df.iloc[i]['close'] and min(df.loc[i-watchleft:i-1,'up_reversal'])<df.iloc[i]['open']:
            up_values = df['up_reversal'].iloc[i-watchleft:i-1]
            up_values = up_values[up_values != 100_000]
            if len(up_values)>1:
                index = up_values.idxmin()
                # stop = df.iloc[i+1]['open']-(df.iloc[i+1]['open']-df.iloc[i]['low'])/2
                stop = df.iloc[i]['low']
                buy_orders.append((df.iloc[i+1]['open'], stop,
                round(df.iloc[i+1]['open'] + (df.iloc[i+1]['open']-stop)*multiplier,2),
                i+1))
                # fig = go.Figure(data=[go.Candlestick(x=df['date'][i-watchleftchart:i+watchrightchart],
                #                             open=df['open'][i-watchleftchart:i+watchrightchart],
                #                             high=df['high'][i-watchleftchart:i+watchrightchart],
                #                             low=df['low'][i-watchleftchart:i+watchrightchart],
                #                             close=df['close'][i-watchleftchart:i+watchrightchart])])   
                # fig.add_trace(go.Scatter(x=[df.iloc[i+1]['date']],y=[df.iloc[i]['close']],mode='markers', marker=dict(size=4, color='white')))
                # fig.add_trace(go.Scatter(x=[df.iloc[i+1]['date']],y=[stop],mode='markers', marker=dict(size=4, color='red')))
                # fig.add_trace(go.Scatter(x=[df.iloc[i+1]['date']],y=[round(df.iloc[i+1]['open'] + (df.iloc[i+1]['open']-stop)*multiplier,2)],mode='markers', marker=dict(size=4, color='green')))
                # fig.add_trace(go.Scatter(x=[df.iloc[index]['date']],y=[df.iloc[index]['low']],mode='markers', marker=dict(size=4, color='blue')))
                # counter+=1
                # pio.write_image(fig, f'screens\\screen№{counter}-1.jpg')
                
                # fig = go.Figure(data=[go.Candlestick(x=dflowertf['date'][i*TFchange-5:i*TFchange+50],
                #                             open=dflowertf['open'][i*TFchange-5:i*TFchange+50],
                #                             high=dflowertf['high'][i*TFchange-5:i*TFchange+50],
                #                             low=dflowertf['low'][i*TFchange-5:i*TFchange+50],
                #                             close=dflowertf['close'][i*TFchange-5:i*TFchange+50])])   
                # pio.write_image(fig, f'screens\\screen№{counter}-2.jpg')
    
    if max(df.loc[i-watchleft:i-1,'down_reversal'])<=df.iloc[i]['high'] and max(df.loc[i-5:i-1,'high'])<df.iloc[i]['high']:
        if max(df.loc[i-watchleft:i-1,'down_reversal'])>df.iloc[i]['close'] and max(df.loc[i-watchleft:i-1,'down_reversal'])>df.iloc[i]['open']:
            down_values = df['down_reversal'].iloc[i-watchleft:i-1]
            down_values = down_values[down_values != 0]
            if len(down_values)>1:
                index = down_values.idxmax()
                # stop = df.iloc[i+1]['open']+(df.iloc[i]['high']-df.iloc[i+1]['open'])/2
                stop = df.iloc[i]['high']
                sell_orders.append((df.iloc[i+1]['open'], stop,
                round(df.iloc[i+1]['open'] - (stop-df.iloc[i+1]['open'])*multiplier,2),
                i+1)) 
                # fig = go.Figure(data=[go.Candlestick(x=df['date'][i-watchleftchart:i+watchrightchart],
                #                             open=df['open'][i-watchleftchart:i+watchrightchart],
                #                             high=df['high'][i-watchleftchart:i+watchrightchart],
                #                             low=df['low'][i-watchleftchart:i+watchrightchart],
                #                             close=df['close'][i-watchleftchart:i+watchrightchart])])   
                # fig.add_trace(go.Scatter(x=[df.iloc[i+1]['date']],y=[df.iloc[i]['close']],mode='markers', marker=dict(size=4, color='white')))
                # fig.add_trace(go.Scatter(x=[df.iloc[i+1]['date']],y=[stop],mode='markers', marker=dict(size=4, color='red')))
                # fig.add_trace(go.Scatter(x=[df.iloc[i+1]['date']],y=[round(df.iloc[i+1]['open'] - (stop-df.iloc[i+1]['open'])*multiplier,2)],mode='markers', marker=dict(size=4, color='green')))
                # fig.add_trace(go.Scatter(x=[df.iloc[index]['date']],y=[df.iloc[index]['high']],mode='markers', marker=dict(size=4, color='blue')))
                # counter+=1
                # pio.write_image(fig, f'screens\\screen№{counter}-1.jpg')
                
                # fig = go.Figure(data=[go.Candlestick(x=dflowertf['date'][i*TFchange-5:i*TFchange+50],
                #                             open=dflowertf['open'][i*TFchange-5:i*TFchange+50],
                #                             high=dflowertf['high'][i*TFchange-5:i*TFchange+50],
                #                             low=dflowertf['low'][i*TFchange-5:i*TFchange+50],
                #                             close=dflowertf['close'][i*TFchange-5:i*TFchange+50])])   
                # pio.write_image(fig, f'screens\\screen№{counter}-2.jpg')









# buy_orders = []
# sell_orders = []

# multiplier = 3
# watchleft = 3
# for i in range(watchleftchart, len(df)-watchrightchart):
#     if min(df.loc[i-watchleft:i-1,'up_reversal'])>=df.iloc[i]['low'] and min(df.loc[i-5:i-1,'low'])>df.iloc[i]['low']:
#         if min(df.loc[i-watchleft:i-1,'up_reversal'])<df.iloc[i]['close'] and min(df.loc[i-watchleft:i-1,'up_reversal'])<df.iloc[i]['open']:
            # buy_orders.append((df.iloc[i+1]['open'], df.iloc[i]['low'],
            # round(df.iloc[i+1]['open'] + (df.iloc[i+1]['open']-df.iloc[i]['low'])*multiplier,2),
            # i+1))
                    
#     if max(df.loc[i-watchleft:i-1,'down_reversal'])<=df.iloc[i]['high'] and max(df.loc[i-5:i-1,'high'])<df.iloc[i]['high']:
#         if max(df.loc[i-watchleft:i-1,'down_reversal'])>df.iloc[i]['close'] and max(df.loc[i-watchleft:i-1,'down_reversal'])>df.iloc[i]['open']:
            # sell_orders.append((df.iloc[i+1]['open'], df.iloc[i]['high'],
            # round(df.iloc[i+1]['open'] - (df.iloc[i]['high']-df.iloc[i+1]['open'])*multiplier,2),
            # i+1)) 

                 
# print('Переходим к бектесту')

backtest(df,buy_orders,sell_orders,100,1,depositchart=False,savecharts=False)

# for order in buy_orders:
    # fig.add_trace(go.Scatter(x=[order[3]],y=[order[0]],mode='markers', marker=dict(size=4, color='white')))
    # fig.add_trace(go.Scatter(x=[order[3]],y=[order[1]],mode='markers', marker=dict(size=4, color='red')))
    # fig.add_trace(go.Scatter(x=[order[3]],y=[order[2]],mode='markers', marker=dict(size=4, color='green')))

# for order in sell_orders:
#     fig.add_trace(go.Scatter(x=[order[3]],y=[order[0]],mode='markers', marker=dict(size=4, color='white')))
#     fig.add_trace(go.Scatter(x=[order[3]],y=[order[1]],mode='markers', marker=dict(size=4, color='red')))
#     fig.add_trace(go.Scatter(x=[order[3]],y=[order[2]],mode='markers', marker=dict(size=4, color='green')))


# for i in range(3, len(df)):
#     if hammer_up(df.loc[i-1, ['open', 'high', 'low', 'close']]):
#         fig.add_trace(go.Scatter(x=[df.iloc[i-1]['date']],y=[df.iloc[i-1]['low']],mode='markers', marker=dict(size=4, color='green')))
#         # fig.add_vline(x=df_higher_tf.iloc[i-1]['date'],line=dict(color='green'))

#     if falling_star_down(df.loc[i-1, ['open', 'high', 'low', 'close']]):
#         fig.add_trace(go.Scatter(x=[df.iloc[i-1]['date']],y=[df.iloc[i-1]['high']],mode='markers', marker=dict(size=4, color='red')))
#         # fig.add_vline(x=df_higher_tf.iloc[i-1]['date'],line=dict(color='red'))

# fig = go.Figure(data=[go.Candlestick(x=df['date'],
#                                       open=df['open'],
#                                       high=df['high'],
#                                       low=df['low'],
#                                       close=df['close'])])
# y = [i*k+b for i in range(64,120)]
# fig.add_trace(go.Scatter(x=df.loc[64:120,'date'], y=y, mode='lines', name='New'))
# for i in range(3, len(df)):
#     if df.iloc[i]['up_reversal'] != 100_000:
#         fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['up_reversal']],mode='markers', marker=dict(size=4, color='green')))
        
#     if df.iloc[i]['down_reversal']:
#         fig.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['down_reversal']],mode='markers', marker=dict(size=4, color='red')))

# # RSX = go.Figure(go.Scatter(
#         x=df['date'],
#         y=df['RSX']
#     ))
# for i in range(3, len(df)):
#     if df.iloc[i]['up_classic_divergence']:
#         RSX.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['RSX']],mode='markers', marker=dict(size=4, color='green')))
        
#     if df.iloc[i]['down_classic_divergence']:
#         RSX.add_trace(go.Scatter(x=[df.iloc[i]['date']],y=[df.iloc[i]['RSX']],mode='markers', marker=dict(size=4, color='red')))


# fig.update_layout(showlegend=False)
# fig.show()
# fig1.update_layout(showlegend=False)
# fig1.show()
# fig2.update_layout(showlegend=False)
# fig2.show()
# RSX.update_layout(showlegend=False)
# RSX.show()


        Total trades: 107,
        Total win trades: 25
        Total lose trades: 82
        Total buy trades: 48
        Total sell trades: 59
        Winrate: 23.36%
        Winrate buy trades: 22.92%
        Winrate sell trades: 23.73%
        Risk: 1%
        Avg RR: 3.0
        Start deposit: 100$
        Deposit: 91.64$
        Max Drawdown(%): 25.13%
        Max Drawdown($): 27.68$
